# Importing necessary packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Generating data 


## Function to be fitted

In [2]:
def function(x):
  y = x*np.sin(.5*x*np.sin(.5*x*np.cos(x)))
  return y
vf = np.vectorize(function)

### Plot the function

In [3]:
init = -2*np.pi
endval = 2*np.pi
step = .00001
x = np.arange(init,endval+step,step)
y = vf(x) + .1*np.random.rand(np.size(x))
#plt.plot(x,y)

## Randomizing and divinding data into test and train sets

In [4]:
p = np.random.permutation(len(x)) # Create a random permutation 
x_perm = x[p]
y_perm = y[p] # Aplling same permutation on both

### Plotting to see if nothing changed (sceptical people go :D )

In [5]:
#plt.plot(x,y)
#plt.plot(x_perm,y_perm,',r')

### Train set and test set, train set is 80% of the data

In [6]:
x_train_full , x_test , y_train_full, y_test = x_perm[:int(np.floor(.8*len(x)))], x_perm[int(np.floor(.8*len(x))):], y_perm[:int(np.floor(.8*len(x)))], y_perm[int(np.floor(.8*len(x))):]

#### Since all the training data is mixed the plot is the same (except for the blank spots)

In [7]:
#plt.plot(x_train_full,y_train_full,',r')

In [8]:
#plt.plot(x_test,y_test,',b')

### Validation and training set

In [9]:
x_train, x_valid, y_train, y_valid = x_train_full[:int(np.floor(.8*len(x)))], x_train_full[int(np.floor(.8*len(x))):], y_train_full[:int(np.floor(.8*len(x)))], y_train_full[int(np.floor(.8*len(x))):]

# Making the DNN 

#### Using keras functional api to construct the the  problem
#### Unsing wide and deep neural net

In [10]:
input_ = keras.layers.Input(shape=(1,))
hidden1 = keras.layers.Dense(30, activation="tanh")(input_)
hidden2 = keras.layers.Dense(30, activation="tanh")(hidden1)
concat = keras.layers.Concatenate()([input_,hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

## Compiling the DNN

In [11]:
model.compile(loss="mse", optimizer=keras.optimizers.Adam())

## Fitting the DNN

In [12]:
history = model.fit(x_train,y_train, epochs=20, validation_data=(x_valid,y_valid))

Epoch 1/20
31416/31416 [==============================] - 19s 604us/step - loss: 2.9578
Epoch 2/20
31416/31416 [==============================] - 18s 569us/step - loss: 0.4358
Epoch 3/20
31416/31416 [==============================] - 19s 590us/step - loss: 0.0579
Epoch 4/20
31416/31416 [==============================] - 18s 579us/step - loss: 0.0035
Epoch 5/20
31416/31416 [==============================] - 18s 567us/step - loss: 0.0031
Epoch 6/20
31416/31416 [==============================] - 19s 617us/step - loss: 0.0028
Epoch 7/20
31416/31416 [==============================] - 17s 553us/step - loss: 0.0028
Epoch 8/20
31416/31416 [==============================] - 17s 545us/step - loss: 0.0026
Epoch 9/20
31416/31416 [==============================] - 17s 528us/step - loss: 0.0026
Epoch 10/20
31416/31416 [==============================] - 16s 510us/step - loss: 0.0026
Epoch 11/20
31416/31416 [==============================] - 17s 532us/step - loss: 0.0024
Epoch 12/20
31416/31416 [=====

In [13]:
mse_test = model.evaluate(x_test, y_test)

7854/7854 [==============================] - 3s 394us/step - loss: 0.0022


In [14]:
x_new = np.arange(init,endval+step/100,step/100)
y_new = model.predict(x_new)
plt.plot(x_new,y_new,'*b')
plt.plot(x,y,'r')